# Limpeza dos microdados do ENEM 2023
- Neste notebook, realizarei a limpeza dos microdados do ENEM 2023.
- Os microdados se constituem no menor nível de desagregação de dados recolhidos por pesquisas, avaliações e exames realizados. No caso do ENEM, os dados estão por participante.
- A limpeza de dados é necessária para performar a análise exploratória de dados. Dado o alto volume de dados e a sua origem (dados reais), algumas tarefas devem ser realizadas:
    -  Identificação e tratamento de valores nulos e duplicados, de acordo com os objetivos da análise.
    -  Remoção de variáveis irrelevantes para a análise.
    -  Feature engineering: Criação e alteração de variáveis existentes. Aqui, irei fundir, remover e renomear categorias com base na melhor formatação para o meu objetivo. Além disso, converter colunas para o tipo de dado correto também será importante.
    -  Otimização de memória: Conversão de variáveis a tipos de dados menores, a fim de melhorar a performance, possibilitando a leitura e manipulação dos dados em menor tempo, sem que haja a perda de informação.
- Irei efetuar duas análises após a limpeza. O objetivo de cada uma delas guiará decisões tomadas futuramente neste notebook.
- Na Análise de Desempenho, tenho como foco analisar o perfil de candidatos que obtêm determinadas notas, quais variáveis se relacionam com as notas e como estas poderiam ser utilizadas. Portanto, é importante utilizar apenas dados de estudantes que estavam presentes em ambos os dias de prova, que de fato obtiveram um resultado. Incluir todos os alunos introduziria assimetrias e distorções na análise.
- Na Análise de Abstenção, tenho como foco analisar quais os fatores que influenciam a ausência do candidato. Portanto, é necessário incluir todos os alunos.

#### 1. Importando as bibliotecas

In [37]:
# Data manipulation and visualization.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# File handling.
import os

#### 2. Coletando os dados
- Considerando o alto volume de dados, irei ler o dataset microdados em "chunks", partes menores, unindo tudo ao fim. Isso tornará a leitura mais eficiente.

In [38]:
# Reading large data in separate chunks, then concatening into a single dataframe again.
chunk_size = 50_000
chunks = []
microdados_path = "../input/microdados_enem_2023/DADOS/MICRODADOS_ENEM_2023.csv"
for chunk in pd.read_csv(microdados_path, sep=';', encoding='ISO-8859-1', chunksize=chunk_size):
   chunks.append(chunk)

microdados = pd.concat(chunks, ignore_index=True)

#### 3. Entendimento inicial dos dados e limpeza dos dados
- Nesta etapa irei observar superficialmente os dados, obtendo dimensões, tipos de dados das variáveis, valores nulos e duplicados, estatísticas descritivas, entre outros.
- Será realizada a limpeza deles também. Irei remover colunas desnecessárias, converter variáveis para os tipos de dado corretos, reduzir o tamanho do dataset, tratar valores nulos e outliers, entre outras tarefas.

#### 3.1 Dicionário de variáveis
- O dicionário de variáveis encontra-se em 'enem-2023/enem-2023/input/microdados_enem_2023/DICIONÁRIO/Dicionário_Microdados_Enem_2023.xlsx'

#### 3.2 Informações gerais sobre os dados

In [39]:
microdados.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210059085136,2023,14,M,2,1,1,1,17,1,...,C,C,B,B,A,B,B,A,A,B
1,210059527735,2023,12,M,2,1,0,1,16,1,...,B,A,B,B,A,A,C,A,D,B
2,210061103945,2023,6,F,1,1,1,1,0,1,...,B,A,A,B,A,A,A,A,A,B
3,210060214087,2023,2,F,1,3,1,2,0,2,...,A,A,A,B,A,A,D,A,A,B
4,210059980948,2023,3,F,1,3,1,2,0,2,...,A,A,A,B,A,A,B,A,A,A


In [40]:
microdados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933955 entries, 0 to 3933954
Data columns (total 76 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   NU_INSCRICAO            int64  
 1   NU_ANO                  int64  
 2   TP_FAIXA_ETARIA         int64  
 3   TP_SEXO                 object 
 4   TP_ESTADO_CIVIL         int64  
 5   TP_COR_RACA             int64  
 6   TP_NACIONALIDADE        int64  
 7   TP_ST_CONCLUSAO         int64  
 8   TP_ANO_CONCLUIU         int64  
 9   TP_ESCOLA               int64  
 10  TP_ENSINO               float64
 11  IN_TREINEIRO            int64  
 12  CO_MUNICIPIO_ESC        float64
 13  NO_MUNICIPIO_ESC        object 
 14  CO_UF_ESC               float64
 15  SG_UF_ESC               object 
 16  TP_DEPENDENCIA_ADM_ESC  float64
 17  TP_LOCALIZACAO_ESC      float64
 18  TP_SIT_FUNC_ESC         float64
 19  CO_MUNICIPIO_PROVA      int64  
 20  NO_MUNICIPIO_PROVA      object 
 21  CO_UF_PROVA             int64  

In [41]:
print(f'O dataset possui {microdados.shape[0]} linhas e {microdados.shape[1]} colunas.')

O dataset possui 3933955 linhas e 76 colunas.


#### 3.3 Valores nulos e duplicados

In [42]:
microdados['NU_INSCRICAO'].duplicated().sum()
microdados.duplicated().sum()

0

In [43]:
null_df = (microdados.isna().sum() / len(microdados) * 100).to_frame().rename(columns={0: 'null_pct'})
null_df['null_count'] = microdados.isna().sum()
null_df.sort_values(by=['null_pct'], ascending=False).head(35)

,null_pct,null_count
TP_LOCALIZACAO_ESC,75.635054,2975449
CO_UF_ESC,75.635054,2975449
TP_SIT_FUNC_ESC,75.635054,2975449
TP_DEPENDENCIA_ADM_ESC,75.635054,2975449
SG_UF_ESC,75.635054,2975449
CO_MUNICIPIO_ESC,75.635054,2975449
NO_MUNICIPIO_ESC,75.635054,2975449
TP_ENSINO,65.960948,2594874
NU_NOTA_CN,31.559283,1241528
CO_PROVA_MT,31.559283,1241528


- Não há observações duplicadas no dataset.
- Variáveis referentes à escola possuem um alto percentual de nulos, acima de 70%, e portanto deverão ser removidas. 
- É possível ver um padrão nas variáveis referentes à nota e código de prova, possuindo o mesmo percentual de nulos para cada dia de aplicação (32% para as provas de ciências da natureza e matemática e 28.2% para as provas de ciências humanas, linguagens e suas tecnologias e redação). Provavelmente esses valores nulos representam alunos que não compareceram ou que foram eliminados, vamos investigar adiante.

In [44]:
# Search for null values in the days when the student was not present for the second day of the exam.
segundo_dia = ['CO_PROVA_CN',
 'NU_NOTA_CN',
 'TX_RESPOSTAS_CN',
 'TX_GABARITO_CN',
 'CO_PROVA_MT',
 'NU_NOTA_MT',
 'TX_RESPOSTAS_MT',
 'TX_GABARITO_MT']
microdados[segundo_dia].loc[(microdados['TP_PRESENCA_MT'] == 0) & (microdados['TP_PRESENCA_CN'] == 0)].isna().sum()

CO_PROVA_CN        1239316
NU_NOTA_CN         1239316
TX_RESPOSTAS_CN    1239316
TX_GABARITO_CN     1239316
CO_PROVA_MT        1239316
NU_NOTA_MT         1239316
TX_RESPOSTAS_MT    1239316
TX_GABARITO_MT     1239316
dtype: int64

In [45]:
# Search for null values in the days when the student was eliminated for the second day of the exam.
microdados[segundo_dia].loc[(microdados['TP_PRESENCA_MT'] == 2) & (microdados['TP_PRESENCA_CN'] == 2)].isna().sum()

CO_PROVA_CN        2212
NU_NOTA_CN         2212
TX_RESPOSTAS_CN    2212
TX_GABARITO_CN     2212
CO_PROVA_MT        2212
NU_NOTA_MT         2212
TX_RESPOSTAS_MT    2212
TX_GABARITO_MT     2212
dtype: int64

In [46]:
# Search for null values in the days when the student was not present for the first day of the exam.
primeiro_dia = ['CO_PROVA_CH', 'CO_PROVA_LC', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_GABARITO_CH',
       'TX_GABARITO_LC', 'TP_STATUS_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2',
       'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'NU_NOTA_REDACAO']
microdados[primeiro_dia].loc[(microdados['TP_PRESENCA_CH'] == 0) & (microdados['TP_PRESENCA_LC'] == 0)].isna().sum()

CO_PROVA_CH          1106714
CO_PROVA_LC          1106714
NU_NOTA_CH           1106714
NU_NOTA_LC           1106714
TX_RESPOSTAS_CH      1106714
TX_RESPOSTAS_LC      1106714
TX_GABARITO_CH       1106714
TX_GABARITO_LC       1106714
TP_STATUS_REDACAO    1106714
NU_NOTA_COMP1        1106714
NU_NOTA_COMP2        1106714
NU_NOTA_COMP3        1106714
NU_NOTA_COMP4        1106714
NU_NOTA_COMP5        1106714
NU_NOTA_REDACAO      1106714
dtype: int64

In [47]:
# Search for null values in the days when the student was eliminated for the first day of the exam.
microdados[primeiro_dia].loc[(microdados['TP_PRESENCA_CH'] == 2) & (microdados['TP_PRESENCA_LC'] == 2)].isna().sum()

CO_PROVA_CH          4598
CO_PROVA_LC          4598
NU_NOTA_CH           4598
NU_NOTA_LC           4598
TX_RESPOSTAS_CH      4598
TX_RESPOSTAS_LC      4598
TX_GABARITO_CH       4598
TX_GABARITO_LC       4598
TP_STATUS_REDACAO    4598
NU_NOTA_COMP1        4598
NU_NOTA_COMP2        4598
NU_NOTA_COMP3        4598
NU_NOTA_COMP4        4598
NU_NOTA_COMP5        4598
NU_NOTA_REDACAO      4598
dtype: int64

In [48]:
microdados['TP_PRESENCA_LC'].value_counts(normalize=True) * 100

TP_PRESENCA_LC
1    71.75077
0    28.13235
2     0.11688
Name: proportion, dtype: float64

In [49]:
microdados['TP_PRESENCA_MT'].value_counts(normalize=True) * 100

TP_PRESENCA_MT
1    68.440717
0    31.503055
2     0.056228
Name: proportion, dtype: float64

- É possível perceber que todas as observações que contêm valores nulos em variáveis contendo informações sobre as provas ocorrem por não comparecimento ou eliminação do estudante.
- Considerando que variáveis contendo informações sobre o status da redacao, código da prova, gabaritos e respostas serão removidas porque não são relevantes para a análise, não imputarei valores nulos nelas.
- Entre os principais objetivos da Análise de Desempenho, estão, entender o perfil dos candidatos que tiram determinadas notas e quais variáveis se relacionam com as notas. Portanto, candidatos faltantes que possuem notas nulas serão separados futuramente. Por enquanto, imputarei essas observações com nota zero. Isso será útil para analisar distribuições e fatores que influenciam o comparecimento (ou não) em cada dia de prova, compondo a Análise de Abstenção.
- Para candidatos que foram eliminados, os quais também possuem nota nula nas provas, irei dropá-los de antemão, pois representam uma parcela muito pequena da população e não trazem informação relevante.

In [50]:
clean_microdados = microdados.copy()

In [51]:
# Drop eliminated records.
eliminated_records = clean_microdados.loc[(clean_microdados['TP_PRESENCA_CN'] == 2) 
                                          | (clean_microdados['TP_PRESENCA_CH'] == 2) 
                                          | (microdados['TP_PRESENCA_LC'] == 2) 
                                          | (microdados['TP_PRESENCA_MT'] == 2)].index
clean_microdados = clean_microdados.drop(eliminated_records)

# Impute the grade as zero for those who didn't do the respective exam day.
to_impute_grade = ['NU_NOTA_CN', 'NU_NOTA_MT', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_REDACAO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5']
clean_microdados[to_impute_grade] = clean_microdados[to_impute_grade].fillna(0)

#### 3.4 Removendo colunas desnecessárias
- É possível perceber que temos um grande número de colunas. Observando o dicionário de variáveis, muitas dessas não nos interessam, e portanto, serão removidas. Entram nesse grupo:
    - Variáveis informando códigos (códigos de prova para cada área do conhecimento, de cidades, unidades federativas, entre outros).
    - Variáveis que podem introduzir viés na análise, como as que identificam cor e raça.
    - Variáveis com gabaritos e respostas de questões.
    - Variáveis altamente desbalanceadas e que podem ser refletidas em valores de outras colunas, como o status da redação.
    - Variáveis informando nacionalidade, número de inscrição e municipio de prova. 
    - Variáveis com um identificador único para cada estudante.
    - Variáveis com alto percentual de valores nulos, como visto acima.
    - Variáveis irrelevantes para a análise e modelagem, como as notas das competências de redação.
- Em resumo: Variáveis com alto percentual de valores nulos, variáveis com variância constante, variáveis com alta cardinalidade, variáveis com um valor por registro, variáveis irrelevantes.

In [52]:
# Defining columns to drop.
to_drop = ['NU_INSCRICAO', 'NU_ANO', 'NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 'TP_NACIONALIDADE', 'TP_COR_RACA', 
           'CO_MUNICIPIO_ESC', 'NO_MUNICIPIO_ESC', 'CO_UF_ESC', 'SG_UF_ESC', 'TP_DEPENDENCIA_ADM_ESC', 
           'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_ENSINO', 'TP_STATUS_REDACAO', 'CO_MUNICIPIO_PROVA', 'CO_UF_PROVA', 
           'CO_PROVA_CN', 'CO_PROVA_CH', 'CO_PROVA_LC', 'CO_PROVA_MT', 'TX_RESPOSTAS_CN', 'TX_RESPOSTAS_CH', 
           'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN', 'TX_GABARITO_CH', 'TX_GABARITO_LC', 
           'TX_GABARITO_MT', 'NO_MUNICIPIO_PROVA', ]
clean_microdados = clean_microdados.drop(columns=to_drop)
clean_microdados.shape

(3927146, 43)

- Ótimo! Já foi possível reduzir o número de variáveis de 76 para 43!

#### 3.5 Alterando variáveis
- De forma geral, variáveis de natureza categórica estão com tipos numéricos. Irei convertê-las com os respectivos valores categóricos a fim de tornar a análise interpretável e de fácil entendimento. Ao mesmo tempo em que converto, irei unir/dropar categorias, de acordo com o melhor critério.
- Irei também padronizar os valores e renomear colunas para melhor entendimento.

In [53]:
# standardize column names to lower 
clean_microdados.columns = [x.lower() for x in clean_microdados.columns]

In [54]:
# remove prefixes from column names
prefixes_to_remove = ['tp_', 'in_', 'sg_', 'nu_', 'no_']
clean_microdados.columns = clean_microdados.columns.to_series().replace(to_replace='^(' + '|'.join(prefixes_to_remove) + ')', value='', regex=True)

In [55]:
# rename questionary columns
to_rename = {'q001': 'escolaridade_pai', 
             'q002': 'escolaridade_mae',
             'q003': 'ocupacao_pai',
             'q004': 'ocupacao_mae',
             'q005': 'numero_pessoas_em_casa',
             'q006': 'renda_familiar_mensal',
             'q007': 'dias_empregado_domestico_em_casa',
             'q008': 'numero_banheiros_em_casa',
             'q009': 'numero_quartos_em_casa',
             'q010': 'numero_carros_em_casa',
             'q011': 'numero_motos_em_casa',
             'q012': 'numero_geladeiras_em_casa',
             'q013': 'numero_freezers_em_casa',
             'q014': 'numero_lavadoras_em_casa',
             'q015': 'numero_secadoras_em_casa',
             'q016': 'numero_microondas_em_casa',
             'q017': 'numero_lavadoras_de_loucas_em_casa',
             'q018': 'possui_aspirador_de_po_em_casa',
             'q019': 'numero_tvs_em_casa',
             'q020': 'possui_dvd_em_casa',
             'q021': 'possui_tv_assinatura_em_casa',
             'q022': 'numero_celulares_em_casa',
             'q023': 'possui_telefone_fixo_em_casa',
             'q024': 'numero_computadores_em_casa',
             'q025': 'possui_acesso_a_internet_em_casa',
             'st_conclusao': 'status_conclusao_ensino_medio'}
clean_microdados = clean_microdados.rename(columns=to_rename)

In [56]:
# map ocupacao categories
ocupacao_mapping = {
    'A': 'Agropecuária',
    'B': 'Serviços Gerais',
    'C': 'Produção e Operações',
    'D': 'Técnicos e Pequenos Negócios',
    'E': 'Profissionais e Liderança',
    'F': 'Não Informado'
}

# Aplicar mapeamento
clean_microdados['ocupacao_pai'] = clean_microdados['ocupacao_pai'].replace(ocupacao_mapping)
clean_microdados['ocupacao_mae'] = clean_microdados['ocupacao_mae'].replace(ocupacao_mapping)

In [57]:
# map faixa etaria variable categories
faixa_etaria_mapping = {
    'Adolescente (< 18)': [1, 2],                  
    'Jovem (18-20)': [3, 4, 5],
    'Jovem adulto (20-24)': [6, 7, 8, 9], 
    'Adulto jovem (25-35)': [10, 11, 12],           
    'Adulto de meia idade (36-45)': [13, 14],       
    'Meia idade a aposentadoria (46-65)': [15, 16, 17, 18],    
    'Idoso (> 66)': [19, 20]                        
}
replaced_faixa_etaria = dict()
for group, keys in faixa_etaria_mapping.items():
    for key in keys:
        replaced_faixa_etaria[key] = group
clean_microdados['faixa_etaria'] = clean_microdados['faixa_etaria'].replace(to_replace=replaced_faixa_etaria)

In [58]:
# map estado civil categories
estado_civil_mapping = {
    0: 'Não informado',
    1: 'Solteiro(a)',
    2: 'Casado(a)/União Estável',
    3: 'Divorciado(a)/Separado(a)',
    4: 'Viúvo(a)'
}
clean_microdados['estado_civil'] = clean_microdados['estado_civil'].replace(estado_civil_mapping)

In [59]:
# map high school conclusion situation categories
st_conclusao_mapping = {
    1: 'Concluído',
    2: 'Último ano',
    3: 'Cursando',
    4: 'Não concluído'
}
clean_microdados['status_conclusao_ensino_medio'] = clean_microdados['status_conclusao_ensino_medio'].replace(st_conclusao_mapping)

In [60]:
# map school type categories
escola_mapping = {
    1: 'Não respondeu',
    2: 'Pública',
    3: 'Privada'
}
clean_microdados['escola'] = clean_microdados['escola'].replace(escola_mapping)

In [61]:
# map presenca categories
clean_microdados[['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt']] = clean_microdados[['presenca_cn', 'presenca_ch', 'presenca_lc', 'presenca_mt']].replace({1: 'Presente', 0: 'Ausente'})

In [62]:
# map lingua categories
clean_microdados['lingua'] = clean_microdados['lingua'].replace({0: 'Inglês', 1: 'Espanhol'})

In [63]:
# map escolaridade categories
escolaridade_mapping = {
    'A': 'Nunca estudou',
    'B': 'Ensino fundamental incompleto',
    'C': 'Ensino fundamental incompleto',
    'D': 'Ensino fundamental completo',
    'E': 'Ensino médio completo',
    'F': 'Ensino superior completo',
    'G': 'Pós-graduação',
    'H': 'Não sei'
}
clean_microdados['escolaridade_pai'] = clean_microdados['escolaridade_pai'].replace(escolaridade_mapping)
clean_microdados['escolaridade_mae'] = clean_microdados['escolaridade_mae'].replace(escolaridade_mapping)

In [64]:
# map renda categories
renda_mapping = {
    'A': 'Nenhuma Renda',
    'B': 'Até R$ 1.320,00',
    'C': 'R$ 1.320,01 - R$ 1.980,00',
    'D': 'R$ 1.980,01 - R$ 2.640,00',
    'E': 'R$ 2.640,01 - R$ 3.300,00',
    'F': 'R$ 3.300,01 - R$ 3.960,00',
    'G': 'R$ 3.960,01 - R$ 5.280,00',
    'H': 'R$ 5.280,01 - R$ 6.600,00',
    'I': 'R$ 6.600,01 - R$ 7.920,00',
    'J': 'R$ 7.920,01 - R$ 9.240,00',
    'K': 'R$ 9.240,01 - R$ 10.560,00',
    'L': 'R$ 10.560,01 - R$ 11.880,00',
    'M': 'R$ 11.880,01 - R$ 13.200,00',
    'N': 'R$ 13.200,01 - R$ 15.840,00',
    'O': 'R$ 15.840,01 - R$ 19.800,00',
    'P': 'R$ 19.800,01 - R$ 26.400,00',
    'Q': 'Acima de R$ 26.400,00'
}
clean_microdados['renda_familiar_mensal'] = clean_microdados['renda_familiar_mensal'].replace(renda_mapping)

In [65]:
# map treineiro categories
treineiro_mapping = {0: 'Não', 1: 'Sim'}
clean_microdados['treineiro'] = clean_microdados['treineiro'].replace(treineiro_mapping)

In [66]:
# map nao_sim categories
nao_sim_mapping = {'A': 'Não', 'B': 'Sim'}
nao_sim_columns = ['possui_aspirador_de_po_em_casa', 'possui_dvd_em_casa', 'possui_tv_assinatura_em_casa',
                   'possui_telefone_fixo_em_casa', 'possui_acesso_a_internet_em_casa']
for feature in nao_sim_columns:
    clean_microdados[feature] = clean_microdados[feature].replace(nao_sim_mapping)

In [67]:
# map dias_empregado_domestico_em_casa and zero to four variables categories
escala_nao_a_quatro_mais_mapping = {
    'A': 'Zero',
    'B': 'Um',
    'C': 'Dois',
    'D': 'Três',
    'E': 'Quatro ou mais'
}
nao_a_quatro_mais_cols = ['numero_banheiros_em_casa', 'numero_quartos_em_casa', 'numero_carros_em_casa', 'numero_motos_em_casa', 'numero_geladeiras_em_casa',
                          'numero_freezers_em_casa', 'numero_lavadoras_em_casa', 'numero_secadoras_em_casa', 'numero_microondas_em_casa', 'numero_lavadoras_de_loucas_em_casa',
                          'numero_tvs_em_casa', 'numero_celulares_em_casa', 'numero_computadores_em_casa', 'dias_empregado_domestico_em_casa']
for feature in nao_a_quatro_mais_cols:
    clean_microdados[feature] = clean_microdados[feature].replace(escala_nao_a_quatro_mais_mapping)

#### 3.6 Otimização de memória
- Irei converter os tipos das variáveis para tipos que consomem menos memória sem perder informação, a fim de otimizar a análise e manipulação dos dados. 
- Após isso irei salvar o dataset resultante em um formato parquet, uma vez que o csv não mantém os tipos convertidos. 
- Uma vez que as notas vão de 0 a 1000 e têm precisão de 1 casa decimal, converterei-as de float64 para float32. Não será utilizado float16 pois o pyarrow não suporta esse tipo e não será possível salvar em parquet.
- A única variável int é número_pessoas_em_casa. Esta será convertida de int64 para int8, sem perder informação, uma vez que ela guarda valores de 1 a 20. Através do tipo int8 podemos representar números de -128 a 127.
- Variáveis com um número limitado de categorias exclusivas serão convertidas para category.

In [68]:
# Obtain and convert columns to lower memory consumption data types.
to_float32 = clean_microdados.select_dtypes('float64').columns.tolist()
to_int8 = 'numero_pessoas_em_casa'
to_category = clean_microdados.select_dtypes('object').columns.tolist()

clean_df = clean_microdados.copy()
clean_df[to_float32] = clean_df[to_float32].astype('float32')
clean_df[to_category] = clean_df[to_category].astype('category')
clean_df[to_int8] = clean_df[to_int8].astype('int8')

In [69]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3927146 entries, 0 to 3933954
Data columns (total 43 columns):
 #   Column                              Dtype   
---  ------                              -----   
 0   faixa_etaria                        category
 1   sexo                                category
 2   estado_civil                        category
 3   status_conclusao_ensino_medio       category
 4   ano_concluiu                        int64   
 5   escola                              category
 6   treineiro                           category
 7   uf_prova                            category
 8   presenca_cn                         category
 9   presenca_ch                         category
 10  presenca_lc                         category
 11  presenca_mt                         category
 12  nota_cn                             float32 
 13  nota_ch                             float32 
 14  nota_lc                             float32 
 15  nota_mt                             f

In [70]:
clean_df.sample(5)

,faixa_etaria,sexo,estado_civil,status_conclusao_ensino_medio,ano_concluiu,escola,treineiro,uf_prova,presenca_cn,presenca_ch,...,numero_microondas_em_casa,numero_lavadoras_de_loucas_em_casa,possui_aspirador_de_po_em_casa,numero_tvs_em_casa,possui_dvd_em_casa,possui_tv_assinatura_em_casa,numero_celulares_em_casa,possui_telefone_fixo_em_casa,numero_computadores_em_casa,possui_acesso_a_internet_em_casa
3856199,Adulto jovem (25-35),F,Casado(a)/União Estável,Concluído,13,Não respondeu,Não,RJ,Presente,Presente,...,Um,Zero,Sim,Um,Não,Não,Dois,Não,Dois,Sim
2334552,Jovem (18-20),M,Solteiro(a),Concluído,2,Não respondeu,Não,SP,Presente,Presente,...,Um,Zero,Sim,Um,Não,Não,Dois,Não,Um,Sim
2278603,Adulto jovem (25-35),M,Divorciado(a)/Separado(a),Concluído,0,Não respondeu,Não,BA,Ausente,Ausente,...,Zero,Zero,Não,Um,Não,Não,Um,Não,Um,Sim
2795262,Jovem (18-20),F,Solteiro(a),Concluído,1,Não respondeu,Não,PB,Presente,Presente,...,Um,Zero,Não,Um,Não,Não,Quatro ou mais,Não,Dois,Sim
2140605,Adulto jovem (25-35),M,Solteiro(a),Concluído,7,Não respondeu,Não,PA,Presente,Presente,...,Um,Zero,Sim,Três,Não,Sim,Quatro ou mais,Não,Dois,Sim


In [71]:
# Convert the memory optimized data to a parquet file in order to maintain the converted data types.
path = '../input/data/clean_df.parquet'
clean_df.to_parquet(path, index=False)

In [72]:
# Read the memory optimized data.
path = '../input/data/clean_df.parquet'
df = pd.read_parquet(path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3927146 entries, 0 to 3927145
Data columns (total 43 columns):
 #   Column                              Dtype   
---  ------                              -----   
 0   faixa_etaria                        category
 1   sexo                                category
 2   estado_civil                        category
 3   status_conclusao_ensino_medio       category
 4   ano_concluiu                        int64   
 5   escola                              category
 6   treineiro                           category
 7   uf_prova                            category
 8   presenca_cn                         category
 9   presenca_ch                         category
 10  presenca_lc                         category
 11  presenca_mt                         category
 12  nota_cn                             float32 
 13  nota_ch                             float32 
 14  nota_lc                             float32 
 15  nota_mt                         

#### 4. Conclusão
- Excelente! Através da limpeza dos dados foi possível reduzir o tamanho do dataset de +2 GB para +243.4 MB, quase 10%! Agora poderemos realizar a análise e manipulação dos dados de forma eficiente.
- Tarefas realizadas:
    -  Identificação e tratamento de valores nulos e duplicados, de acordo com os objetivos da análise.
    -  Remoção de variáveis irrelevantes para a análise. Variáveis com alto percentual de valores nulos, variáveis com variância constante, variáveis com alta cardinalidade, variáveis com um valor por registro, variáveis inúteis.
    -  Feature engineering: Criação e alteração de variáveis existentes. Aqui, irei fundir, remover e renomear categorias com base na melhor formatação para o meu objetivo. Além disso, converter colunas para o tipo de dado correto também será importante.
    -  Otimização de memória: Conversão de variáveis a tipos de dados menores, a fim de melhorar a performance, possibilitando a leitura e manipulação dos dados em menor tempo, sem que haja a perda de informação.